In [ ]:
rm(list = ls())
library(Seurat)
library(monocle3)
library(dplyr)
library(BiocParallel)
library(ggplot2)
library(qs)

In [ ]:
seurat.data = qread("/home/user/ALI/Macrophages/Outdata/Sub_annotion.qs")

In [ ]:
# seurat.data  <- subset(seurat.data, subset = celltype %in% c("Macrophages_c2", "Macrophages_c9", "Macrophages_c10","Macrophages_c5"))

In [ ]:
levels(seurat.data)

In [ ]:
scRNA <- subset(seurat.data, subset = !grepl("\\(", celltype))


In [ ]:
levels(scRNA)

In [ ]:
DimPlot(scRNA,pt.size = 0.8,group.by = "celltype",label = T)

In [ ]:
levels(Idents(scRNA)) #打出来细胞类型供复制
# [1] "CD4+ T-cells"      "Fibroblasts"       "B-cells"          
# [4] "CD8+ T-cells"      "Neutrophils"       "Monocytes"        
# [7] "Adipocytes"        "NK cells"          "Endothelial cells"
 
# 重设等级也可以不设
# scRNA$celltype <- factor(scRNA$celltype,
#                          levels = c("Endothelial cells"))
 
Idents(scRNA) <- scRNA$celltype

In [ ]:
expression_matrix <- GetAssayData(scRNA, assay = 'RNA',layer  = 'counts')
cell_metadata <- scRNA@meta.data 
gene_annotation <- data.frame(gene_short_name = rownames(expression_matrix))
rownames(gene_annotation) <- rownames(expression_matrix)
cds <- new_cell_data_set(expression_matrix,
                         cell_metadata = cell_metadata,
                         gene_metadata = gene_annotation)

In [ ]:
cds <- preprocess_cds(cds, num_dim = 50)

In [ ]:
cds <- reduce_dimension(cds)
plot_cells(cds, label_groups_by_cluster=FALSE,  color_cells_by = "celltype")

In [ ]:
# sub_cds <- cds[, colData(cds)$celltype %in% c("Macrophages_c5", "Macrophages_c2")]

In [ ]:
cds = qread("/home/guoliming/Brown/ALI_Gaoji/sc_bulk_analysis_Epithelial/6_monocle2/Macrophages_pseudotime.qs")

In [ ]:
# options(repr.plot.width=6, repr.plot.height=6)  # 单位英寸
pdf("maerk_Clec4e.pdf",width = 8,height = 8)
ciliated_genes <- c("Clec4e")

p <- plot_cells(cds,cell_size=0.8,
           genes=ciliated_genes,
           label_cell_groups=FALSE,
           show_trajectory_graph=FALSE)

p + theme(
  legend.title = element_text(size=16),   # 图例标题
  legend.text = element_text(size=16),    # 图例文本
  axis.text = element_text(size=20),      # 坐标轴文本
  axis.title = element_text(size=20),      # 坐标轴标题
plot.title  = element_text(size = 20, hjust = 1), # 主标题
)& 
  NoAxes() &  
  theme(
    legend.title   = element_text(size = 16),   # 图例标题
    legend.text    = element_text(size = 16),   # 图例文字
    legend.key.size = unit(1, "cm")
  )
dev.off()

In [ ]:
library(patchwork)  # 或 cowplot

pdf("maerk_M1.pdf", width = 20, height = 12)
ciliated_genes <- c("Nos2","Arg1","Mrc1","Cd86","Cxcl9","Cxcl10","Chil3","Cd163")

plots <- lapply(ciliated_genes, function(g) {
  plot_cells(
    cds,
    genes = g,
    cell_size = 0.8,
    label_cell_groups = FALSE,
    show_trajectory_graph = FALSE
  ) + 
    theme(
      legend.title   = element_text(size=16),
      legend.text    = element_text(size=16),
      axis.text      = element_text(size=16),
      axis.title     = element_text(size=16),
      plot.title     = element_blank(),       # 去掉标题
      legend.key.size = unit(1, "cm")
    ) +
    NoAxes()
})

# 每行4列排版
p <- wrap_plots(plots, ncol = 4)
print(p)

dev.off()


In [ ]:
pdf("maerk_M1.pdf",width = 12,height = 8)
ciliated_genes <- c("Nos2","Arg1","Mrc1","Cd86","Cxcl9","Cxcl10","Chil3","Cd163")

p <- plot_cells(cds,
           genes=ciliated_genes,cell_size=0.8,
           label_cell_groups=FALSE,
           show_trajectory_graph=FALSE)

p + theme(
  legend.title = element_text(size=16),   # 图例标题
  legend.text = element_text(size=16),    # 图例文本
  axis.text = element_text(size=16),      # 坐标轴文本
  axis.title = element_text(size=16),      # 坐标轴标题
plot.title  = element_text(size = 16), # 主标题
)& 
  NoAxes() &  
  theme(
    legend.title   = element_text(size = 16),   # 图例标题
    legend.text    = element_text(size = 16),   # 图例文字
    legend.key.size = unit(1, "cm")
  )
dev.off()

In [ ]:
cds <- cluster_cells(cds)
plot_cells(cds, color_cells_by = "partition")

In [ ]:
cds <- learn_graph(cds)
plot_cells(cds,
           color_cells_by = "celltype",
           label_groups_by_cluster=FALSE,
           label_leaves=FALSE,
           label_branch_points=FALSE,
            trajectory_graph_segment_size = 1)

In [ ]:
library(ggplot2)

pdf("Macrophages_pseudotime_celltype.pdf", width=8, height=8)
p <- plot_cells(cds,
                color_cells_by = "celltype",cell_size=0.8,
                label_groups_by_cluster=FALSE,
                label_leaves=FALSE,
                label_branch_points=FALSE,
                # graph_label_size=20,
                trajectory_graph_segment_size = 1.5)  # 节点标签字体

p + theme(
  legend.title = element_text(size=16),   # 图例标题
  legend.text = element_text(size=16),    # 图例文本
  axis.text = element_text(size=16),      # 坐标轴文本
  axis.title = element_text(size=16),      # 坐标轴标题
      # legend.title   = element_text(size = 16),   # 图例标题
    # legend.text    = element_text(size = 16)   # 图例文字
     
)& NoAxes()

dev.off()

In [ ]:
table(colData(cds)$celltype)

In [ ]:
# a helper function to identify the root principal points:
get_earliest_principal_node <- function(cds, time_bin="Macrophages_c5"){
  cell_ids <- which(colData(cds)[, "celltype"] == time_bin)
  
  closest_vertex <-
  cds@principal_graph_aux[["UMAP"]]$pr_graph_cell_proj_closest_vertex
  closest_vertex <- as.matrix(closest_vertex[colnames(cds), ])
  root_pr_nodes <-
  igraph::V(principal_graph(cds)[["UMAP"]])$name[as.numeric(names
  (which.max(table(closest_vertex[cell_ids,]))))]
  
  root_pr_nodes
}
cds <- order_cells(cds, root_pr_nodes=get_earliest_principal_node(cds))

In [ ]:
# options(repr.plot.width=6, repr.plot.height=6)  # 单位英寸
pdf("Macrophages_pseudotime_5.pdf",width = 10,height = 8)
p <- plot_cells(cds,
           color_cells_by = "pseudotime",cell_size=0.8,
           label_cell_groups=TRUE,
           label_leaves=FALSE,
           label_branch_points=FALSE,
           graph_label_size=4,trajectory_graph_segment_size = 1.5)
p + theme(
  legend.title = element_text(size=16),   # 图例标题
  legend.text = element_text(size=16),    # 图例文本
  axis.text = element_text(size=16),      # 坐标轴文本
  axis.title = element_text(size=16),      # 坐标轴标题
      # legend.title   = element_text(size = 16),   # 图例标题
    # legend.text    = element_text(size = 16)   # 图例文字
) & 
  NoAxes() &  
  theme(
    legend.title   = element_text(size = 16),   # 图例标题
    legend.text    = element_text(size = 16),   # 图例文字
  )
dev.off()

In [ ]:
qsave(cds,"Macrophages_pseudotime.qs")